In [2]:
# Here we are importing the necessary libraries
from __future__ import print_function
from __future__ import division
import numpy as np
import torch
import torch.nn.functional as F
import torchvision
import matplotlib.pyplot as plt
import numpy as np
import time
import os
import copy
import torch.optim as optim
#This is for Google Collab Notebook - it doesn't have torchplot installed!
#!pip install torchplot
#import torchplot as tplt

from torch import nn
from torch import optim
from torchvision import datasets, models, transforms
from torchvision import transforms as T
from tqdm import tqdm
from torch.utils.data import DataLoader, sampler, random_split

print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("We will be using the:", device)

In [3]:
emnist_dataset = datasets.EMNIST(root='./', # here
                                split='letters',
                                train=True, # train split
                                download=True, # we want to get the data
                                transform=T.ToTensor(), # put it into tensor format
                              )
train_data = DataLoader(emnist_dataset,
                        batch_size=10,
                        )

In [5]:
import matplotlib.pyplot as plt
data = iter(train_data) # Let's iterate on it
single_point = next(data)
ToPIL = T.ToPILImage() # Converting function
img0 = ToPIL(single_point[0][0])
img1 = ToPIL(single_point[0][1])
# Plotting
fig, axs = plt.subplots(1,2)
axs[0].imshow(img0)
axs[1].imshow(img1)

In [6]:
class LeNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6,5,padding=2)
        self.conv2 = nn.Conv2d(6,16,5, padding=0)
        self.fc1 = nn.Linear(5*5*16, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84,62)
        
    def forward(self, x):
        x = torch.sigmoid(self.conv1(x))
        x =  nn.MaxPool2d(kernel_size=2, stride=2)(x)
        x = torch.sigmoid(self.conv2(x))
        x =  nn.MaxPool2d(kernel_size=2, stride=2)(x)
        
        x = x.view(-1, 5*5*16)
        
        x = torch.sigmoid(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        x = self.fc3(x)
        return x

In [7]:
#https://medium.com/analytics-vidhya/alexnet-a-simple-implementation-using-pytorch-30c14e8b6db2
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        #self.conv0 = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=1, stride=1, padding=0)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels= 96, kernel_size= 11, stride=4, padding=0 )
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2)
        self.conv2 = nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, stride= 1, padding= 2)
        self.conv3 = nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, stride= 1, padding= 1)
        self.conv4 = nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, stride=1, padding=1)
        self.conv5 = nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.fc1  = nn.Linear(in_features= 9216, out_features= 4096)
        self.fc2  = nn.Linear(in_features= 4096, out_features= 4096)
        self.fc3 = nn.Linear(in_features=4096 , out_features=62)


    def forward(self,x):
        #x = F.relu(self.conv0(x))
        x = F.relu(self.conv1(x))
        x = self.maxpool(x)
        x = F.relu(self.conv2(x))
        x = self.maxpool(x)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = self.maxpool(x)
        x = x.reshape(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
# Let's set up some parameters
learning_rate=0.001
nepochs = 10
ninputs=1*28*28
nout=62

#model = AlexNet().to(device)
model = LeNet().to(device)

print(model)

# We need an optimizer that tells us what form of gradient descent to do
optimizer = torch.optim.Adam(model.parameters(), learning_rate)
#optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate, momentum=0.9,weight_decay=5e-4)

# We also need a loss function
LossFunction = nn.CrossEntropyLoss()

batch_size = 64

#creating a dinstinct transform class for the train, validation and test dataset
tranform_train = transforms.Compose([
    transforms.Grayscale(num_output_channels=3), 
    transforms.Resize((227,227)),     
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225])])

emnist_dataset = datasets.EMNIST(root='./', split='byclass', train=True, download=True, transform=T.ToTensor(),)
train_data = DataLoader(emnist_dataset, batch_size=batch_size,)

emnist_test_dataset = datasets.EMNIST(root='./', split='byclass', train=False, download=False, transform=T.ToTensor(),)
test_data = DataLoader(emnist_test_dataset, batch_size=batch_size,)


# This is default on but let's just be pedantic
model.train()
loss_history = []
test_accuracy_history = []
train_accuracy_history = []
loss = torch.Tensor([0])
for epoch in tqdm(range(nepochs), desc=f"Epoch", unit="epoch", disable=False):
    accuracy = 0
    for (data, label) in tqdm(train_data, desc="iteration", unit="%", disable=True):
        # Here we clear the gradients
        optimizer.zero_grad(set_to_none=True)
        
        # We need to make sure the tensors are on the same device as our model
        data = data.to(device)
        label = label.to(device)
        out = model(data)
        
        loss = LossFunction(out, label)
        
        loss.backward() # This function calculates all our gradients
        optimizer.step() # This function does our gradient descent with those gradients
        loss_history.append(loss.item())
        answers = out.max(dim=1)[1]
        accuracy += (answers == label).sum()
    # Append the training accuracy
    accuracy = accuracy / len(emnist_dataset)*100
    train_accuracy_history.append(accuracy)
    
    #let's get the test accuracy to see how well it generalizes
    test_accuracy = 0
    for (data, label) in test_data:
        data = data.to(device)
        label = label.to(device)
        out = model(data)
        answers = out.max(dim=1)[1]
        test_accuracy += (answers == label).sum()
    # Append the testing accuracy
    test_accuracy = test_accuracy / len(emnist_test_dataset)*100
    test_accuracy_history.append(test_accuracy)
    #Print the results
    print(f"Epoch: {epoch} \n Loss: {loss.item()} \n Train Accuracy: {accuracy}% \n Test Accuracy: {test_accuracy}%")


In [11]:
# Plot the loss per iteration
    plt.plot(loss_history)
    plt.title("Neural Network Loss")
    plt.xlabel("Number of iterations")
    plt.ylabel("Loss")
    plt.show()
    # Plot the accuracy per epoch
    tplt.plot(test_accuracy_history, 'r')
    tplt.plot(train_accuracy_history, 'b')
    tplt.title("Neural Network Accuracy")
    tplt.xlabel("Epoch Number")
    tplt.ylabel("Accuracy")
    tplt.show()

In [ ]:
#Some sources - DON'T DELETE

#https://www.nist.gov/itl/products-and-services/emnist-dataset#:~:text=The%20EMNIST%20Letters%20dataset%20merges,with%20the%20original%20MNIST%20dataset.
#https://pytorch.org/vision/stable/datasets.html#emnist